# Analyze captions and try to get list of nouns of different images.
* Is object from segmentation in the caption?
* Analyze nouns in captions to see if they correlate with coco object names

## Start with filtered dataset with `person` class

In [51]:
import json
import pickle
import random
import collections
import nltk
nltk.download('averaged_perceptron_tagger')
COCO_PATH = "../../datasets/coco"

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/braian/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [25]:
# load processed file
with open('task1_person_accessory_data.json', "r") as f:
    filtered_task1_data = json.load(f)
    
# category name map
with open('filename.pickle', 'rb') as handle:
    category_id_map = pickle.load(handle)

In [26]:
filtered_task1_data.keys()

dict_keys(['27', '28', '31', '32', '33'])

In [45]:
category_id_map[27]

'backpack'

In [29]:
caption = filtered_task1_data['33'][0]['annotations']['caption']
caption

'A photo of a person on a motorcycle on the road.'

In [30]:
word_tokens = nltk.word_tokenize(caption)
print(word_tokens)
tag = nltk.pos_tag(word_tokens)
print(tag)

['A', 'photo', 'of', 'a', 'person', 'on', 'a', 'motorcycle', 'on', 'the', 'road', '.']
[('A', 'DT'), ('photo', 'NN'), ('of', 'IN'), ('a', 'DT'), ('person', 'NN'), ('on', 'IN'), ('a', 'DT'), ('motorcycle', 'NN'), ('on', 'IN'), ('the', 'DT'), ('road', 'NN'), ('.', '.')]


In [31]:
[token for token, pos_tag in tag if pos_tag == 'NN']

['photo', 'person', 'motorcycle', 'road']

In [68]:
[(category_id_map[int(obj_id)], obj_id) for obj_id in filtered_task1_data]

[('backpack', '27'),
 ('umbrella', '28'),
 ('handbag', '31'),
 ('tie', '32'),
 ('suitcase', '33')]

### extract all nouns from captions, save as a list 
Nouns for a particular image should match the list of distinct categories in segmentation

In [66]:
word_counts = {}
for obj_class in filtered_task1_data:    
    noun_list = []
    for idx, ann in enumerate(filtered_task1_data[obj_class]):
        caption = ann['annotations']['caption']
        tokens = nltk.word_tokenize(caption)
        tag = nltk.pos_tag(tokens)
        ann['annotations']["caption_nouns"] = [token for token, pos_tag in tag if pos_tag == 'NN']
        noun_list.extend(ann['annotations']["caption_nouns"])
        if random.random() > 0.9:
            print(f"[{idx, category_id_map[int(obj_class)]}] caption : {caption}")           
            print(f"[{idx, category_id_map[int(obj_class)]}] capt. noun : {sorted(ann['annotations']['caption_nouns'])}")
            print(f"[{idx, category_id_map[int(obj_class)]}] segm. obj  : {sorted([category_id_map[obj_id] for obj_id in ann['annotations']['distinct_obj']])}")
    word_counts[obj_class] = collections.Counter(noun_list)

[(12, 'backpack')] caption : Man toilet seat while playing a game on computer.
[(12, 'backpack')] capt. noun : ['computer', 'game', 'seat']
[(12, 'backpack')] segm. obj  : ['backpack', 'book', 'keyboard', 'mouse', 'person', 'teddy bear', 'toilet', 'tv']
[(17, 'backpack')] caption : Two women, a child, and a young man with Down's syndrome on a picnic blanket with food while people sit in folding chairs in the background in a park.
[(17, 'backpack')] capt. noun : ['background', 'blanket', 'child', 'food', 'man', 'park', 'picnic', 'syndrome']
[(17, 'backpack')] segm. obj  : ['apple', 'backpack', 'bottle', 'chair', 'handbag', 'person', 'teddy bear']
[(25, 'backpack')] caption : A little girl in a sunhat is standing near other people.
[(25, 'backpack')] capt. noun : ['girl', 'sunhat']
[(25, 'backpack')] segm. obj  : ['backpack', 'bicycle', 'bottle', 'car', 'frisbee', 'person', 'umbrella']
[(26, 'backpack')] caption : a road busy with some people on bikes and some kids walking 
[(26, 'backpa

[(539, 'backpack')] caption : a number of people on a bus near luggage bags 
[(539, 'backpack')] capt. noun : ['bus', 'luggage', 'number']
[(539, 'backpack')] segm. obj  : ['backpack', 'bus', 'car', 'person', 'suitcase']
[(545, 'backpack')] caption : A red truck and a huge pile of trash in the side of a road
[(545, 'backpack')] capt. noun : ['pile', 'road', 'side', 'trash', 'truck']
[(545, 'backpack')] segm. obj  : ['backpack', 'car', 'person', 'suitcase', 'truck']
[(553, 'backpack')] caption : A woman sitting in an airport next to a  bag of luggage.
[(553, 'backpack')] capt. noun : ['airport', 'bag', 'luggage', 'woman']
[(553, 'backpack')] segm. obj  : ['backpack', 'handbag', 'person', 'suitcase', 'teddy bear']
[(558, 'backpack')] caption : Young couple smiling and standing in front of a baggage claim sign at the airport.
[(558, 'backpack')] capt. noun : ['airport', 'baggage', 'claim', 'couple', 'front', 'sign']
[(558, 'backpack')] segm. obj  : ['backpack', 'bench', 'handbag', 'person

[(1079, 'backpack')] caption : A man riding a skateboard next to a man standing up against a fence.
[(1079, 'backpack')] capt. noun : ['fence', 'man', 'man', 'skateboard']
[(1079, 'backpack')] segm. obj  : ['backpack', 'bench', 'dog', 'fire hydrant', 'motorcycle', 'person']
[(1082, 'backpack')] caption : Two people on bicycles crossing the road with a dog.
[(1082, 'backpack')] capt. noun : ['dog', 'road']
[(1082, 'backpack')] segm. obj  : ['backpack', 'bicycle', 'car', 'couch', 'dog', 'person', 'truck']
[(1087, 'backpack')] caption : a man kneeling and playing with a dog 
[(1087, 'backpack')] capt. noun : ['dog', 'kneeling', 'man']
[(1087, 'backpack')] segm. obj  : ['backpack', 'bottle', 'dog', 'person']
[(1105, 'backpack')] caption : A snowboarder sits in snow beside a dog.
[(1105, 'backpack')] capt. noun : ['dog', 'snowboarder']
[(1105, 'backpack')] segm. obj  : ['backpack', 'dog', 'person', 'snowboard']
[(1125, 'backpack')] caption : A little boy walking across a hard wood floor.
[(

[(1615, 'backpack')] caption : Two different times are displayed in the bus stop.
[(1615, 'backpack')] capt. noun : ['bus', 'stop']
[(1615, 'backpack')] segm. obj  : ['backpack', 'bicycle', 'bus', 'person']
[(1631, 'backpack')] caption : Pedestrians near intersection with traffic in large city.
[(1631, 'backpack')] capt. noun : ['city', 'intersection', 'traffic']
[(1631, 'backpack')] segm. obj  : ['backpack', 'bicycle', 'car', 'cell phone', 'handbag', 'person', 'traffic light', 'truck']
[(1664, 'backpack')] caption : Bicyclists wait on an urban street corner in front of a bus.
[(1664, 'backpack')] capt. noun : ['bus', 'corner', 'front', 'street']
[(1664, 'backpack')] segm. obj  : ['backpack', 'bicycle', 'bus', 'car', 'handbag', 'person', 'traffic light']
[(1686, 'backpack')] caption : A bicyclist stands looking across a bridge over a river going through a city.
[(1686, 'backpack')] capt. noun : ['bicyclist', 'bridge', 'city', 'river']
[(1686, 'backpack')] segm. obj  : ['backpack', 'bic

[(2161, 'backpack')] caption : A group of men standing around a cake on a table.
[(2161, 'backpack')] capt. noun : ['cake', 'group', 'table']
[(2161, 'backpack')] segm. obj  : ['backpack', 'bottle', 'bowl', 'cake', 'cell phone', 'dining table', 'fork', 'person', 'spoon']
[(2166, 'backpack')] caption : Two individuals on bikes pointing to an another individual on an elephant on a roadway. 
[(2166, 'backpack')] capt. noun : ['elephant', 'individual', 'roadway']
[(2166, 'backpack')] segm. obj  : ['backpack', 'bicycle', 'elephant', 'person', 'truck']
[(2182, 'backpack')] caption : A woman uses her cellphone while riding on the bus.
[(2182, 'backpack')] capt. noun : ['bus', 'cellphone', 'woman']
[(2182, 'backpack')] segm. obj  : ['backpack', 'book', 'cell phone', 'handbag', 'person', 'tie', 'train']
[(2198, 'backpack')] caption : A parking lot outside a large business, where a man loads boxes on a trailer
[(2198, 'backpack')] capt. noun : ['business', 'lot', 'man', 'parking', 'trailer']
[(2

...

[(4587, 'handbag')] caption : A woman with a handbag holding a giant teddy bear
[(4587, 'handbag')] capt. noun : ['bear', 'handbag', 'teddy', 'woman']
[(4587, 'handbag')] segm. obj  : ['chair', 'handbag', 'person', 'teddy bear']
[(4590, 'handbag')] caption : a few people that are walking around in the middle of a road
[(4590, 'handbag')] capt. noun : ['middle', 'road']
[(4590, 'handbag')] segm. obj  : ['backpack', 'cell phone', 'handbag', 'person', 'skateboard']
[(4602, 'handbag')] caption : four people gather around a sign on a city street
[(4602, 'handbag')] capt. noun : ['city', 'sign', 'street']
[(4602, 'handbag')] segm. obj  : ['bench', 'car', 'handbag', 'person']
[(4603, 'handbag')] caption : A man with blonde hair holding a stuffed animal
[(4603, 'handbag')] capt. noun : ['animal', 'hair', 'man', 'stuffed']
[(4603, 'handbag')] segm. obj  : ['cup', 'handbag', 'person', 'teddy bear']
[(4606, 'handbag')] caption : A family sharing a meal at a picnic table.
[(4606, 'handbag')] capt.

[(270, 'tie')] caption : A horse that is jumping vertically in front of a wooden pole.
[(270, 'tie')] capt. noun : ['front', 'horse', 'pole']
[(270, 'tie')] segm. obj  : ['bench', 'horse', 'person', 'tie']
[(276, 'tie')] caption : A man riding a horse drown carriage driving across a field.
[(276, 'tie')] capt. noun : ['carriage', 'field', 'horse', 'man']
[(276, 'tie')] segm. obj  : ['bus', 'horse', 'person', 'tie']
[(284, 'tie')] caption : A man wearing green pants and a bow tie skateboards down a road. 
[(284, 'tie')] capt. noun : ['bow', 'man', 'road', 'tie']
[(284, 'tie')] segm. obj  : ['car', 'person', 'skateboard', 'tie', 'truck']
[(300, 'tie')] caption : A man holds and gestures toward a sandwich
[(300, 'tie')] capt. noun : ['man', 'sandwich']
[(300, 'tie')] segm. obj  : ['book', 'person', 'sandwich', 'tie']
[(317, 'tie')] caption : Two boys carrying hot dogs and other snacks at an outdoor sporting event.
[(317, 'tie')] capt. noun : ['event', 'sporting']
[(317, 'tie')] segm. obj 

[(882, 'tie')] caption : Two men and a women holding roses at a political event.
[(882, 'tie')] capt. noun : ['event']
[(882, 'tie')] segm. obj  : ['book', 'dining table', 'person', 'potted plant', 'tie', 'vase']
[(892, 'tie')] caption : Two men sitting in a room by a piano smiling.
[(892, 'tie')] capt. noun : ['piano', 'room', 'smiling']
[(892, 'tie')] segm. obj  : ['chair', 'cup', 'person', 'tie', 'vase']
[(907, 'tie')] caption : Several people carry open umbrellas on a wet city street that is surrounded by tall colorful buildings.
[(907, 'tie')] capt. noun : ['city', 'street']
[(907, 'tie')] segm. obj  : ['bicycle', 'car', 'person', 'tie', 'umbrella']
[(914, 'tie')] caption : A man wearing an orange ribbon is standing in a parking lot.
[(914, 'tie')] capt. noun : ['lot', 'man', 'orange', 'parking', 'ribbon']
[(914, 'tie')] segm. obj  : ['person', 'tie', 'truck', 'umbrella']
[(919, 'tie')] caption : various shapes and sizes of suitcases lying on a floor
[(919, 'tie')] capt. noun : ['

[(1451, 'tie')] caption : A lone kite is flying below a colorful kite stand.
[(1451, 'tie')] capt. noun : ['kite', 'lone', 'stand']
[(1451, 'tie')] segm. obj  : ['handbag', 'kite', 'person', 'tie']
[(1461, 'tie')] caption : A wedding couple is cutting in to their ugly cake.
[(1461, 'tie')] capt. noun : ['cake', 'couple']
[(1461, 'tie')] segm. obj  : ['cake', 'knife', 'person', 'tie']
[(1465, 'tie')] caption : A newly married couple feeding each other cake
[(1465, 'tie')] capt. noun : ['cake', 'couple']
[(1465, 'tie')] segm. obj  : ['cake', 'dining table', 'fork', 'knife', 'person', 'tie']
[(1471, 'tie')] caption : A bride and groom are cutting a wedding cake. 
[(1471, 'tie')] capt. noun : ['bride', 'cake', 'groom']
[(1471, 'tie')] segm. obj  : ['cake', 'dining table', 'knife', 'person', 'tie']
[(1479, 'tie')] caption : Two men in suits feed each other cake.
[(1479, 'tie')] capt. noun : ['cake']
[(1479, 'tie')] segm. obj  : ['cake', 'cup', 'dining table', 'fork', 'knife', 'person', 'tie

[(455, 'suitcase')] caption : A woman walking down a sidewalk lugging a bag of luggage behind her.
[(455, 'suitcase')] capt. noun : ['bag', 'luggage', 'sidewalk', 'woman']
[(455, 'suitcase')] segm. obj  : ['bicycle', 'car', 'clock', 'handbag', 'person', 'suitcase']
[(470, 'suitcase')] caption : A long train crosses a bridge set high above a city street.
[(470, 'suitcase')] capt. noun : ['bridge', 'city', 'street', 'train']
[(470, 'suitcase')] segm. obj  : ['bicycle', 'bus', 'car', 'handbag', 'person', 'suitcase', 'train']
[(475, 'suitcase')] caption : A person crossing the street as a bus turns a corner.
[(475, 'suitcase')] capt. noun : ['bus', 'corner', 'person', 'street']
[(475, 'suitcase')] segm. obj  : ['bus', 'car', 'person', 'potted plant', 'suitcase', 'traffic light', 'truck']
[(478, 'suitcase')] caption : A person is dragging two suitcases behind her and has a bag strapped on her shoulder. 
[(478, 'suitcase')] capt. noun : ['bag', 'person', 'shoulder']
[(478, 'suitcase')] segm.

In [67]:
for obj_id, word_count_obj in word_counts.items():
    print(category_id_map[int(obj_id)], word_count_obj.most_common()[:10])
    print()

backpack [('man', 631), ('street', 485), ('group', 354), ('woman', 299), ('bus', 233), ('train', 198), ('person', 179), ('city', 175), ('front', 146), ('luggage', 140)]

umbrella [('umbrella', 456), ('street', 332), ('man', 294), ('woman', 270), ('beach', 214), ('group', 174), ('rain', 129), ('person', 122), ('city', 109), ('sidewalk', 88)]

handbag [('street', 777), ('woman', 770), ('man', 677), ('group', 456), ('bus', 314), ('city', 312), ('train', 238), ('umbrella', 235), ('front', 204), ('bench', 204)]

tie [('man', 480), ('cake', 165), ('group', 145), ('woman', 130), ('table', 117), ('tie', 109), ('street', 91), ('suit', 89), ('front', 72), ('couple', 64)]

suitcase [('luggage', 233), ('man', 186), ('train', 121), ('woman', 112), ('street', 92), ('airport', 77), ('suitcase', 73), ('group', 68), ('bus', 65), ('station', 62)]



### Add a flag to annotations in which the secondary object appears in the caption
Based on most common nouns above, create a hardcoded list of nouns to filter captions


In [70]:
common_nouns = {
    '27': [],
    '28': ['umbrella', 'rain'],
    '31': [],
    '32': ['tie', 'suit'],
    '33': ['luggage', 'suitcase'],
}

In [71]:
for obj_class in filtered_task1_data:
    for idx, ann in enumerate(filtered_task1_data[obj_class]):
        ann['annotations']['second_object_in_caption'] = False
        # if at least one of the nouns is in caption, set flag to true
        for noun in common_nouns[obj_class]:
            if noun in ann['annotations']["caption_nouns"]:
                ann['annotations']['second_object_in_caption'] = True
                break

In [72]:
# save processed dataset in a json file
with open('task1_person_accessory_data_w_caption.json', "w") as f:
    json.dump(filtered_task1_data, f)

# Now with filtered dataset with `dinning table` class

In [61]:
# load processed file
with open('task2_dining_objects_data.json', "r") as f:
    filtered_task2_data = json.load(f)

In [62]:
word_counts = {}
for obj_class in filtered_task2_data:    
    noun_list = []
    for idx, ann in enumerate(filtered_task2_data[obj_class]):
        caption = ann['annotations']['caption']
        tokens = nltk.word_tokenize(caption)
        tag = nltk.pos_tag(tokens)
        ann['annotations']["caption_nouns"] = [token for token, pos_tag in tag if pos_tag == 'NN']
        noun_list.extend(ann['annotations']["caption_nouns"])
        if random.random() > 0.9:
            print(f"[{idx, category_id_map[int(obj_class)]}] caption : {caption}")           
            print(f"[{idx, category_id_map[int(obj_class)]}] capt. noun : {sorted(ann['annotations']['caption_nouns'])}")
            print(f"[{idx, category_id_map[int(obj_class)]}] segm. obj  : {sorted([category_id_map[obj_id] for obj_id in ann['annotations']['distinct_obj']])}")
    word_counts[obj_class] = collections.Counter(noun_list)

[(7, 'bottle')] caption : The kitchen is has a stainless steal refrigerator. 
[(7, 'bottle')] capt. noun : ['kitchen', 'refrigerator', 'steal']
[(7, 'bottle')] segm. obj  : ['bottle', 'bowl', 'chair', 'dining table', 'orange', 'oven', 'potted plant', 'refrigerator', 'sink']
[(12, 'bottle')] caption : Eating and kitchen area of a small apartment 
[(12, 'bottle')] capt. noun : ['Eating', 'apartment', 'area']
[(12, 'bottle')] segm. obj  : ['book', 'bottle', 'bowl', 'chair', 'clock', 'dining table', 'knife', 'oven', 'potted plant', 'refrigerator', 'sink', 'vase']
[(36, 'bottle')] caption : A bottle of wine sitting next to a wine glass
[(36, 'bottle')] capt. noun : ['bottle', 'glass', 'wine', 'wine']
[(36, 'bottle')] segm. obj  : ['bottle', 'bowl', 'cup', 'dining table', 'knife', 'wine glass']
[(50, 'bottle')] caption : a empty kitchen with some tables and chairs in it 
[(50, 'bottle')] capt. noun : ['kitchen']
[(50, 'bottle')] segm. obj  : ['bottle', 'chair', 'dining table', 'microwave', '

[(614, 'bottle')] caption : This donut has blue, green, and yellow confetti with white frosting on it.
[(614, 'bottle')] capt. noun : ['confetti', 'donut']
[(614, 'bottle')] segm. obj  : ['bottle', 'cup', 'dining table', 'donut']
[(617, 'bottle')] caption : some food lay on a plate next to some drinks 
[(617, 'bottle')] capt. noun : ['food', 'plate']
[(617, 'bottle')] segm. obj  : ['bottle', 'bowl', 'carrot', 'cup', 'dining table', 'fork', 'sandwich']
[(618, 'bottle')] caption : A plate of food with chicken. carrots and bread on top of a wooden table. 
[(618, 'bottle')] capt. noun : ['bread', 'chicken', 'food', 'plate', 'table', 'top']
[(618, 'bottle')] segm. obj  : ['bottle', 'carrot', 'dining table', 'fork', 'knife', 'wine glass']
[(624, 'bottle')] caption : A meal with sandwiches and cola on a counter.
[(624, 'bottle')] capt. noun : ['cola', 'counter', 'meal']
[(624, 'bottle')] segm. obj  : ['bottle', 'cup', 'dining table', 'sandwich']
[(626, 'bottle')] caption : Glasses of champagn

[(69, 'cup')] caption : There are large stacks of kitchenware neatly arranged on the stainless stail counter.
[(69, 'cup')] capt. noun : ['counter', 'kitchenware', 'stail', 'stainless']
[(69, 'cup')] segm. obj  : ['bowl', 'cup', 'dining table', 'fork', 'knife', 'spoon']
[(76, 'cup')] caption : a cup of coffee and a paperback book on top of a table 
[(76, 'cup')] capt. noun : ['book', 'coffee', 'cup', 'paperback', 'table', 'top']
[(76, 'cup')] segm. obj  : ['book', 'cup', 'dining table', 'spoon']
[(81, 'cup')] caption : Two teddy bears sitting at a wooden table having a meal.
[(81, 'cup')] capt. noun : ['meal', 'table', 'wooden']
[(81, 'cup')] segm. obj  : ['bowl', 'cat', 'chair', 'cup', 'dining table', 'teddy bear', 'wine glass']
[(97, 'cup')] caption : The ingredients for a bacon, lettuce, tomato sandwich are on the table.
[(97, 'cup')] capt. noun : ['bacon', 'lettuce', 'sandwich', 'table', 'tomato']
[(97, 'cup')] segm. obj  : ['book', 'bottle', 'chair', 'cup', 'dining table', 'fork',

[(610, 'cup')] caption : A desk with a fish bowl, computer and plant.
[(610, 'cup')] capt. noun : ['bowl', 'computer', 'desk', 'plant']
[(610, 'cup')] segm. obj  : ['bowl', 'chair', 'cup', 'dining table', 'keyboard', 'mouse', 'potted plant', 'scissors', 'tv']
[(625, 'cup')] caption : A large cat is sitting on its owner's lap.
[(625, 'cup')] capt. noun : ['cat', 'lap', 'owner']
[(625, 'cup')] segm. obj  : ['book', 'bowl', 'cat', 'cup', 'dining table', 'fork', 'potted plant', 'spoon', 'wine glass']
[(639, 'cup')] caption : A living room with sofa, chairs, tables, and lamps.
[(639, 'cup')] capt. noun : ['living', 'room', 'sofa']
[(639, 'cup')] segm. obj  : ['chair', 'couch', 'cup', 'dining table', 'potted plant', 'tv']
[(653, 'cup')] caption : A table and chairs is near a couch, door, and window.
[(653, 'cup')] capt. noun : ['couch', 'door', 'table', 'window']
[(653, 'cup')] segm. obj  : ['apple', 'bowl', 'cat', 'chair', 'couch', 'cup', 'dining table', 'laptop', 'orange', 'tv']
[(656, 'cu

[(1083, 'cup')] caption : A white plate with a sandwich that is cut in half
[(1083, 'cup')] capt. noun : ['half', 'plate', 'sandwich']
[(1083, 'cup')] segm. obj  : ['bowl', 'cup', 'dining table', 'fork', 'knife', 'sandwich', 'spoon']
[(1090, 'cup')] caption : A plate of food is near a fork, bowl, and cup of coffee.
[(1090, 'cup')] capt. noun : ['bowl', 'coffee', 'cup', 'food', 'fork', 'plate']
[(1090, 'cup')] segm. obj  : ['apple', 'bowl', 'cup', 'dining table', 'fork', 'sandwich']
[(1105, 'cup')] caption : breakfast on a plate with utensils and drinks

[(1105, 'cup')] capt. noun : ['breakfast', 'plate']
[(1105, 'cup')] segm. obj  : ['cup', 'dining table', 'fork', 'knife', 'sandwich']
[(1113, 'cup')] caption : a sandwich cut in to and two side dishes
[(1113, 'cup')] capt. noun : ['cut', 'side']
[(1113, 'cup')] segm. obj  : ['bottle', 'bowl', 'cup', 'dining table', 'sandwich', 'spoon']
[(1121, 'cup')] caption : A sandwich made on cinnamon swirl bread with a drink
[(1121, 'cup')] capt. n

...

[(2543, 'dining table')] caption : a green sofa and love couch in a living room 
[(2543, 'dining table')] capt. noun : ['couch', 'living', 'room', 'sofa']
[(2543, 'dining table')] segm. obj  : ['book', 'chair', 'clock', 'couch', 'dining table', 'microwave', 'oven', 'vase']
[(2564, 'dining table')] caption : this kitchen has rocks on the counters and wood cabinets
[(2564, 'dining table')] capt. noun : ['kitchen', 'wood']
[(2564, 'dining table')] segm. obj  : ['bowl', 'chair', 'cup', 'dining table', 'microwave', 'oven', 'refrigerator']
[(2566, 'dining table')] caption : A small section of a room shows a portion of a window frame and below it a table with two chairs, one holding a jacket, while on the table rests some large shards of glass, two apples, two bananas, and a long vase.    
[(2566, 'dining table')] capt. noun : ['frame', 'glass', 'jacket', 'portion', 'room', 'section', 'table', 'vase', 'window']
[(2566, 'dining table')] segm. obj  : ['apple', 'banana', 'chair', 'dining table',

[(3141, 'dining table')] caption : A sandwich made on cinnamon swirl bread with a drink
[(3141, 'dining table')] capt. noun : ['bread', 'drink', 'sandwich', 'swirl']
[(3141, 'dining table')] segm. obj  : ['bowl', 'carrot', 'chair', 'cup', 'dining table', 'sandwich', 'spoon', 'wine glass']
[(3145, 'dining table')] caption : A sandwich and mashed potatoes with gravy served on a red plate.
[(3145, 'dining table')] capt. noun : ['plate', 'sandwich']
[(3145, 'dining table')] segm. obj  : ['cup', 'dining table', 'fork', 'sandwich']
[(3150, 'dining table')] caption : A club sandwich with a side of chips sits on a plate.
[(3150, 'dining table')] capt. noun : ['club', 'plate', 'sandwich', 'side']
[(3150, 'dining table')] segm. obj  : ['dining table', 'sandwich', 'spoon']
[(3151, 'dining table')] caption : A sandwich cut in half with pickles on a white plate.
[(3151, 'dining table')] capt. noun : ['cut', 'half', 'plate']
[(3151, 'dining table')] segm. obj  : ['bowl', 'cup', 'dining table', 'fork

[(3651, 'dining table')] caption : A vase on a table with several pink flowers. 
[(3651, 'dining table')] capt. noun : ['table', 'vase']
[(3651, 'dining table')] segm. obj  : ['chair', 'cup', 'dining table', 'vase', 'wine glass']
[(3675, 'dining table')] caption : A white plate include meat, rice and a fried food.
[(3675, 'dining table')] capt. noun : ['food', 'include', 'meat', 'plate', 'rice']
[(3675, 'dining table')] segm. obj  : ['bowl', 'cup', 'dining table', 'fork', 'spoon']
[(3686, 'dining table')] caption : A spoon, a bowl of soup and an orange.
[(3686, 'dining table')] capt. noun : ['bowl', 'orange', 'soup', 'spoon']
[(3686, 'dining table')] segm. obj  : ['bowl', 'carrot', 'dining table', 'orange', 'spoon']
[(3701, 'dining table')] caption : a plate with eaten pancakes and a latte on the side
[(3701, 'dining table')] capt. noun : ['latte', 'plate', 'side']
[(3701, 'dining table')] segm. obj  : ['cake', 'cup', 'dining table', 'spoon']
[(3732, 'dining table')] caption : Three do

[(4148, 'dining table')] caption : A dinner table set with a checkered table clothe. 
[(4148, 'dining table')] capt. noun : ['clothe', 'dinner', 'table', 'table']
[(4148, 'dining table')] segm. obj  : ['apple', 'cup', 'dining table', 'fork', 'knife', 'orange']
[(4152, 'dining table')] caption : A wooden table holding a black tray with various food and tea.
[(4152, 'dining table')] capt. noun : ['food', 'table', 'tea', 'tray']
[(4152, 'dining table')] segm. obj  : ['bowl', 'cake', 'cup', 'dining table']
[(4165, 'dining table')] caption : A box with three foot long hot dogs inside of it.
[(4165, 'dining table')] capt. noun : ['box', 'foot']
[(4165, 'dining table')] segm. obj  : ['bowl', 'cup', 'dining table', 'hot dog']
[(4176, 'dining table')] caption : A salad on top of a pita with a drink on the side
[(4176, 'dining table')] capt. noun : ['drink', 'pita', 'salad', 'side', 'top']
[(4176, 'dining table')] segm. obj  : ['cup', 'dining table', 'pizza']
[(4184, 'dining table')] caption : A

[(4656, 'dining table')] caption : a cup of coffee on a black plate on a table next to a laptop.
[(4656, 'dining table')] capt. noun : ['coffee', 'cup', 'laptop', 'plate']
[(4656, 'dining table')] segm. obj  : ['bowl', 'cup', 'dining table', 'laptop', 'spoon']
[(4658, 'dining table')] caption : a bowl full of some meat, carrots and some lettuce 
[(4658, 'dining table')] capt. noun : ['bowl', 'lettuce', 'meat']
[(4658, 'dining table')] segm. obj  : ['bowl', 'carrot', 'dining table', 'spoon']
[(4688, 'dining table')] caption : Coffee, rolls and noodles are on a table.
[(4688, 'dining table')] capt. noun : ['Coffee', 'table']
[(4688, 'dining table')] segm. obj  : ['cake', 'cup', 'dining table', 'spoon']
[(4706, 'dining table')] caption : A still life photograph of oranges and apples
[(4706, 'dining table')] capt. noun : ['life', 'photograph']
[(4706, 'dining table')] segm. obj  : ['apple', 'bowl', 'dining table', 'orange']
[(4723, 'dining table')] caption : A white plate topped with a san

In [63]:
for obj_id, word_count_obj in word_counts.items():
    print(category_id_map[int(obj_id)], word_count_obj.most_common()[:5])

bottle [('table', 366), ('kitchen', 194), ('plate', 186), ('room', 124), ('food', 113)]
cup [('plate', 622), ('table', 572), ('food', 245), ('pizza', 196), ('sandwich', 162)]
fork [('plate', 615), ('table', 306), ('food', 231), ('pizza', 165), ('sandwich', 114)]
knife [('plate', 425), ('table', 303), ('food', 159), ('pizza', 155), ('kitchen', 87)]
bowl [('table', 559), ('plate', 399), ('food', 320), ('bowl', 299), ('kitchen', 199)]
chair [('table', 650), ('room', 544), ('kitchen', 338), ('living', 268), ('plate', 113)]
dining table [('table', 1469), ('plate', 1283), ('room', 611), ('food', 551), ('kitchen', 415)]
